# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.


Признаки
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак
- Price — цена (евро)

In [14]:
pip install sweetviz

Note: you may need to restart the kernel to use updated packages.


In [51]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import sweetviz as sv
import matplotlib.pyplot as plt

## Подготовка данных

In [16]:
df = pd.read_csv('/datasets/autos.csv')

In [17]:
df.sample(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
278557,2016-04-04 15:54:02,1200,small,1998,manual,44,polo,150000,2,NaN,volkswagen,NaN,2016-04-04 00:00:00,0,57645,2016-04-05 12:09:12
306152,2016-03-13 18:53:22,3200,NaN,2016,manual,108,3_reihe,125000,3,NaN,peugeot,no,2016-03-13 00:00:00,0,33775,2016-03-24 07:15:37
263731,2016-03-22 20:52:39,11900,sedan,2006,auto,224,other,150000,7,gasoline,mercedes_benz,yes,2016-03-22 00:00:00,0,4416,2016-04-02 16:18:07
166869,2016-03-23 11:25:23,2700,small,2006,NaN,0,polo,150000,0,gasoline,volkswagen,NaN,2016-03-23 00:00:00,0,60314,2016-03-23 11:31:48
235520,2016-03-18 15:36:55,2900,convertible,2000,manual,75,lupo,150000,6,petrol,volkswagen,no,2016-03-18 00:00:00,0,89081,2016-04-05 22:18:59


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [19]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Price,354369.0,4416.656776,4514.158514,0.0,1050.0,2700.0,6400.0,20000.0
RegistrationYear,354369.0,2004.234448,90.227958,1000.0,1999.0,2003.0,2008.0,9999.0
Power,354369.0,110.094337,189.850405,0.0,69.0,105.0,143.0,20000.0
Kilometer,354369.0,128211.172535,37905.341530,5000.0,125000.0,150000.0,150000.0,150000.0
RegistrationMonth,354369.0,5.714645,3.726421,0.0,3.0,6.0,9.0,12.0
NumberOfPictures,354369.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
PostalCode,354369.0,50508.689087,25783.096248,1067.0,30165.0,49413.0,71083.0,99998.0


In [20]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

В данных присутствуют пропуски:
- VehicleType:          37490
- Gearbox:              19833
- Model:                19705
- FuelType:             32895
- NotRepaired:          71154

In [21]:
comparison_report = sv.analyze([df, "Car price"])

                                             |          | [  0%]   00:00 -> (? left)

In [49]:
comparison_report.show_notebook((w=900, h=700, scale=0.8)

SyntaxError: invalid syntax (3360956505.py, line 1)

Заменим Null значения в признаках Model и NotRepaired на "unknoun"

In [23]:
df[['Model', 'NotRepaired']] = df[['Model', 'NotRepaired']].fillna('unknoun')

In [24]:
df[['Model', 'NotRepaired']].isna().sum()

Model          0
NotRepaired    0
dtype: int64

Заменим Null значения в столбцах VehicleType, Gearbox, FuelType. Так как данные категориальные, самой логичной заменой является мода этих признаков, сгрупированных по модели (столбец "Model")

In [25]:
def mode(series):
    mode = series.mode()
    if len(mode) > 0:
        return mode[0]
    return None

In [26]:
features = ['VehicleType','Gearbox', 'FuelType']
for feature in tqdm(features):
    df[feature] = df[feature].fillna(df.groupby('Model')[feature].transform(mode))

100%|██████████| 3/3 [00:00<00:00,  3.32it/s]


In [27]:
df.isna().sum()

DateCrawled          0
Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Kilometer            0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
DateCreated          0
NumberOfPictures     0
PostalCode           0
LastSeen             0
dtype: int64

Вывод: Null значения заменены.

 В данных присцтствуют признаки которые не влияют на прогнозируемую стоимость, а именно:

- DateCrawled — дата скачивания анкеты из базы
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя
- NumberOfPictures — количество фотографий автомобиля  
Удалим их   
Дополнительно, переведем DateCreated в формат год и месяц создания анкеты

In [28]:
df['DateCreated'] = pd.to_datetime(df['DateCreated'],format='%Y.%m.%d %H:%M:%S')


In [29]:
df['DateCreatedYear'] = pd.DatetimeIndex(df['DateCreated']).year
df['DateCreatedMonth'] = pd.DatetimeIndex(df['DateCreated']).month

In [30]:
df = df.drop(['DateCrawled', 'PostalCode', 'LastSeen', 'NumberOfPictures', 'DateCreated'], axis=1)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354369 non-null  int64 
 1   VehicleType        354369 non-null  object
 2   RegistrationYear   354369 non-null  int64 
 3   Gearbox            354369 non-null  object
 4   Power              354369 non-null  int64 
 5   Model              354369 non-null  object
 6   Kilometer          354369 non-null  int64 
 7   RegistrationMonth  354369 non-null  int64 
 8   FuelType           354369 non-null  object
 9   Brand              354369 non-null  object
 10  NotRepaired        354369 non-null  object
 11  DateCreatedYear    354369 non-null  int64 
 12  DateCreatedMonth   354369 non-null  int64 
dtypes: int64(7), object(6)
memory usage: 35.1+ MB


Разделим выборку на trein и valid: 

In [32]:
df_train, df_valid = train_test_split(df, train_size=0.75, random_state=12345)
features_train = df_train.drop('Price', axis=1)
target_train = df_train['Price']
features_valid = df_valid.drop('Price', axis=1)
target_valid = df_valid['Price']
print(features_train.shape, target_train.shape, features_valid.shape, target_valid.shape)

(265776, 12) (265776,) (88593, 12) (88593,)


**Вывод по п.1:**  
Данные загружены из csv файла. В вборке - 354369 объкта, достаточное колличество для проведения исследования.  
В данных присутствовали Null значения, которые были заменяны:  в признаках Model и NotRepaired на "unknoun", в признаках VehicleType, Gearbox, FuelType - на моду этих признаков, сгрупированных по модели (столбец "Model")  
Для обучения моделей, датафрейм был разделен на train и valid выборки.

## Обучение моделей

LightGBM

**Исследуем модель LightGBM**

In [33]:
features_train[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']] = features_train[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']].astype('category')
features_valid[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']] = features_valid[['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']].astype('category')

In [34]:
%%time
model = LGBMRegressor()
model.fit(features_train, target_train)

CPU times: user 8.07 s, sys: 90.9 ms, total: 8.16 s
Wall time: 8.24 s


LGBMRegressor()

In [35]:
%%time
predict_cb = model.predict(features_valid)
RMSE_CB = mean_squared_error(target_valid, predict_cb)**0.5
print('RMSE LightGBM на валидационной выборке:', RMSE_CB)

RMSE LightGBM на валидационной выборке: 1787.051214844383
CPU times: user 1.28 s, sys: 317 µs, total: 1.28 s
Wall time: 1.29 s


Вывод: 
модель LightGBM показала следующие результаты:
- время обучения на train выборке - 6.25 сек.
- время предсказания на valid выборке - 904 мсек.
- RMSE на валидационной выборке - 1787

**Исследуем модель CatBoostRegressor**

In [36]:
%%time
model = CatBoostRegressor(loss_function='RMSE', iterations=50)
cat_features=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
model.fit(features_train, target_train, cat_features=cat_features, verbose=10)

Learning rate set to 0.5
0:	learn: 3284.8785243	total: 333ms	remaining: 16.3s
10:	learn: 2045.0147455	total: 2.74s	remaining: 9.73s
20:	learn: 1955.0049921	total: 5.04s	remaining: 6.97s
30:	learn: 1909.5916279	total: 7.38s	remaining: 4.52s
40:	learn: 1877.3699701	total: 9.72s	remaining: 2.13s
49:	learn: 1860.8319420	total: 11.8s	remaining: 0us
CPU times: user 12.2 s, sys: 66.9 ms, total: 12.3 s
Wall time: 13.8 s


In [54]:
model.feature_importances_

array([ 6.37644798, 32.27029327,  1.50987139, 20.42916577,  7.02512071,
       10.12792414,  2.6582799 ,  3.41620591, 11.18991366,  4.58381542,
        0.03463451,  0.37832734])

In [37]:
%%time
predict_cb = model.predict(features_valid)
RMSE_CB = mean_squared_error(target_valid, predict_cb)**0.5
print('RMSE  CatBoostRegressor на валидационной выборке:', RMSE_CB)

RMSE  CatBoostRegressor на валидационной выборке: 1880.2616744310826
CPU times: user 90.9 ms, sys: 146 µs, total: 91.1 ms
Wall time: 92.3 ms


In [38]:
model.get_best_score()

{'learn': {'RMSE': 1860.8319420092646}}

Вывод: 
модель CatBoostRegressor показала следующие результаты:
- время обучения на train выборке - 12,9 сек.
- время предсказания на valid выборке - 215 мсек.
- RMSE на тестовой выборке, полученное кросс-валидацией - 1860,83
- RMSE на валидационной выборке - 1880,23

**Исследуем модель LinearRegression()**

In [39]:
regressor = LinearRegression()
num = ['RegistrationYear', 'Power', 'Kilometer', 'RegistrationMonth', 'DateCreatedYear', 'DateCreatedMonth']
cat = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
transformer = ColumnTransformer(transformers=[('num', StandardScaler(), num),('cat', OneHotEncoder(), cat)])
pipeline = Pipeline(steps=[('t',transformer), ('m',regressor)])

In [40]:
%%time
pipeline.fit(features_train, target_train)

CPU times: user 31.6 s, sys: 28.4 s, total: 60 s
Wall time: 1min


Pipeline(steps=[('t',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['RegistrationYear', 'Power',
                                                   'Kilometer',
                                                   'RegistrationMonth',
                                                   'DateCreatedYear',
                                                   'DateCreatedMonth']),
                                                 ('cat', OneHotEncoder(),
                                                  ['VehicleType', 'Gearbox',
                                                   'Model', 'FuelType', 'Brand',
                                                   'NotRepaired'])])),
                ('m', LinearRegression())])

In [41]:
%%time
predict_lr = pipeline.predict(features_valid)
RMSE_LR = mean_squared_error(target_valid, predict_lr)**0.5
print('RMSE  LinearRegression на валидационной выборке:', RMSE_LR)

RMSE  LinearRegression на валидационной выборке: 3196.008591801047
CPU times: user 214 ms, sys: 9.01 ms, total: 223 ms
Wall time: 239 ms


In [42]:
%%time

RMSE_LR = (cross_val_score(pipeline, features_train, target_train, cv=5, scoring='neg_mean_squared_error').mean()*-1)**0.5



CPU times: user 2min 20s, sys: 2min 21s, total: 4min 42s
Wall time: 4min 42s


In [43]:
RMSE_LR

3197.5843373173457

Вывод: 
модель LinearRegression() показала следующие результаты:
- время обучения на train выборке - 34,7 сек.
- время предсказания на valid выборке - 131 мсек.
- RMSE - 3197.94

## Анализ моделей

**Анализ модель CatBoostRegressor**  
На данном этапе изменим гиперпараметры одели и проанализируем влияние на скорость и качество:

In [44]:
%%time
model = LGBMRegressor(n_estimators=500, num_leaves=50, max_depth=100 )
model.fit(features_train, target_train)

CPU times: user 33.3 s, sys: 0 ns, total: 33.3 s
Wall time: 33.6 s


LGBMRegressor(max_depth=100, n_estimators=500, num_leaves=50)

In [45]:
%%time
predict_cb = model.predict(features_valid)
RMSE_CB = mean_squared_error(target_valid, predict_cb)**0.5
print('RMSE LightGBM на валидационной выборке:', RMSE_CB)

RMSE LightGBM на валидационной выборке: 1698.1308405019197
CPU times: user 7.19 s, sys: 0 ns, total: 7.19 s
Wall time: 7.32 s


**Анализ модель CatBoostRegressor**  
На данном этапе изменим гиперпараметры одели и проанализируем влияние на скорость и качество:

In [46]:
%%time
model = CatBoostRegressor(loss_function='RMSE', iterations=500, depth=10, learning_rate=0.5)
cat_features=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
model.fit(features_train, target_train, cat_features=cat_features, verbose=100)

0:	learn: 3129.3572591	total: 1.39s	remaining: 11m 35s
100:	learn: 1581.3420633	total: 2m 14s	remaining: 8m 50s
200:	learn: 1474.4637127	total: 4m 48s	remaining: 7m 8s
300:	learn: 1400.8874611	total: 7m 17s	remaining: 4m 49s
400:	learn: 1344.4223604	total: 9m 46s	remaining: 2m 24s
499:	learn: 1300.9179589	total: 12m 8s	remaining: 0us
CPU times: user 11min 55s, sys: 0 ns, total: 11min 55s
Wall time: 12min 13s


In [47]:
%%time
predict_cb = model.predict(features_valid)
RMSE_CB = mean_squared_error(target_valid, predict_cb)**0.5
print('RMSE CatBoostRegressor на валидационной выборке:', RMSE_CB)

RMSE CatBoostRegressor на валидационной выборке: 1743.5253105234433
CPU times: user 1.15 s, sys: 0 ns, total: 1.15 s
Wall time: 1.16 s


**Вывод по п3**  

Изменяя гиперпараметры мы  добились улучшение точности предсказания, но при этом увеличивается время обучения: 

Для модели CatBoostRegressor, установив параметры(iterations=500, depth=10, learning_rate=0.5): 
- RMSE изменилась c 1880,23 на 1743,52
- время обучения изменилось с 12,9 сек. на 4 мин. 14 сек.
- время предсказания изменилось с 215 мсек. на 900 мсек

Для модели LinearRegression, установив параметры(n_estimators=500, num_leaves=50, max_depth=100): 
- RMSE изменилась c 1787 на 1698
- время обучения изменилось с 6,25 мсек. на 24,7 сек.
- время предсказания изменилось с 904 мсек. на 4,22 сек


**Общий вывод по проекту**  
1.  Модель LinearRegression() - наихудшая точность предсказания, при этом модели бустинга показывают лучшие результаты при меньшем времени обучения и предсказания.
- время обучения на train выборке - 34,7 сек.
- время предсказания на valid выборке - 131 мсек.
- RMSE - 3197.94
2. Модель CatBoostRegressor - при применении параметров (iterations=500, depth=10, learning_rate=0.5) показала следующие результаты:  
- время обучения на train выборке - 4 мин 14 сек.
- время предсказания на valid выборке - 900 мсек.
- RMSE - 1743,5
3. Модель LightGBM - при применении параметров (n_estimators=500, num_leaves=50, max_depth=100 ) показала следующие результаты:  
- время обучения на train выборке - 24,7 сек.
- время предсказания на valid выборке - 4,22сек.
- RMSE - 1698  

Для модели LightGBM дефолтные настройки параметров показывают наилучшие результаты RMSE за минимальное время обучения и предсказания. Перебирая настройки гиперпараметров можно несущественно улучшить RMSE, однако время обучения и предсказания заметно увеличивается. Это же справидливо и для модели CatBoostRegressor.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей
